In [3]:
import urllib.request
from bs4 import BeautifulSoup
import re
import numpy as np
import datetime
import unicodedata
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time

from tqdm import tqdm_notebook as tqdm
from ipywidgets import IntProgress

In [4]:
pref_list = ['hokkaido','aomori','akita','yamagata','iwate','miyagi', 'fukushima',
             'aichi','gifu','shizuoka','mie','niigata','yamanashi', 'nagano', 
             'ishikawa','toyama','fukui','okayama','hiroshima','tottori','shimane',
             'yamaguchi','kagawa','tokushima','ehime','kochi','tokyo','kanagawa',
             'chiba','tochigi','ibaraki','gunma','osaka','hyogo','kyoto','shiga',
             'nara','wakayama','fukuoka','saga','nagasaki','kumamoto','oita',
             'miyazaki','kagoshima','okinawa']

In [ ]:
contents = []
for pref in tqdm(pref_list):
    
    isContinue = True
    num = 1
    
    content = []
    while(isContinue):
        if(num == 1):
            print('Loading pages of '+pref)
        
        site_url = "https://tabelog.com/"+pref+"/rstLst/"+str(num)+"/?Srt=D&SrtT=rvcn"
        nem = urllib.request.urlopen(site_url).read()
        soup = BeautifulSoup(nem,"html.parser")
        names = soup.find_all("a", class_="list-rst__rst-name-target cpy-rst-name")
        rates = soup.find_all("span", class_="c-rating__val c-rating__val--strong list-rst__rating-val")
        amounts = soup.find_all("em", class_="list-rst__rvw-count-num cpy-review-count")

        amounts = [int(i.text) for i in amounts]

        if(min(amounts) <= 100):
            isContinue = False
            idx = np.argmin(np.abs(np.array(amounts) - 100))
        else:
            idx = False

        if(idx is not False):
            url = [i.get('href') for i in names][:idx]
            name = [i.text for i in names][:idx]
            rates = [float(i.text) for i in rates][:idx]
            amounts = amounts[:idx]
        else:
            url = [i.get('href') for i in names]
            name = [i.text for i in names]
            rates = [float(i.text) for i in rates]

        for n,r,u,a in zip(name,rates,url,amounts):
            dic = {'name': n, 'rates': r, 'amounts': a, 'url': u, 'pref':pref}
            content.append(dic)

        num += 1
        if(num == 60):
            isContinue = False
        time.sleep(1)
        
    contents.append(content)

In [249]:
contents = np.array(contents)
np.save('scraping.npy', contents)

In [2]:
contents = np.load('scraping.npy')

In [ ]:
for n,content in enumerate(tqdm(contents)):
    for m,c in enumerate(tqdm(content)):
        url = c['url']
        isLoad = True
        try:
            nem = urllib.request.urlopen(url).read()
        except:
            isLoad = False
            time.sleep(5)
        if(isLoad == True):
            soup = BeautifulSoup(nem,"html.parser")
            isPremium = soup.find('h3', class_='pr-comment-title js-pr-title')
            if(isPremium is None):
                contents[n][m].update({'isPremium': False})
            else:
                contents[n][m].update({'isPremium': True})
            contents[n][m].update({'isLoad': True})
        else:
            contents[n][m].update({'isPremium': False})
            contents[n][m].update({'isLoad': False})
        time.sleep(1)

In [5]:
contents = np.array(contents)
np.save('total_contents.npy', contents)